# Predicting ALICE

Last Edited: Mar 2024 \
Edited By: John Cox

### Goal
Collate several data sources, including FAFSA completion, high school graduation, and kindergarten readiness, and use them to predict ALICE rates.

### Data sources & descriptions
High school graduation: https://data.wa.gov \
FAFSA completion: https://studentaid.gov/data-center/student/application-volume/fafsa-completion-high-school \
Kindergarten readiness: https://ospi.k12.wa.us/data-reporting/data-portal?title=readiness&field_years1_target_id=All&field_data_domain_target_id=All&field_level_of_aggregation_target_id=All&field_grade_span_target_id=All \
ALICE: https://www.unitedforalice.org/washington

## FAFSA Completion Rates
First, we load in the FAFSA data

In [12]:
# import packages
import pandas as pd
import glob

In [13]:
# get all the fafsa files in the folder
f_all_files = glob.glob('FAFSA raw/*.csv')

f_li = []

# unpack the list of files
for filename in f_all_files:
    f_df = pd.read_csv(filename, index_col=None, header=0, dtype = {'School Code' : str, 'Name' : str, 'City' : str, 
                                                   'State' : str, 'Submitted' : str, 'Completed' : str, 
                                                   'Cycle' : str}
                        , parse_dates=[7])
    f_li.append(f_df)

# put the files all together into one df
fafsa_raw = pd.concat(f_li, axis=0, ignore_index=True)
fafsa_raw.head()

,School Code,Name,City,State,Submitted,Completed,Cycle,As of Date
0,NaN,AKIACHAK SCHOOL,AKIACHAK,AK,<5,<5,2022/2023,2022-03-31
1,NaN,ALASKA MIDDLE COLLEGE SCHOOL,ANCHORAGE,AK,5,5,2022/2023,2022-03-31
2,NaN,ANCHORAGE CHRISTIAN SCHOOLS,ANCHORAGE,AK,15,14,2022/2023,2022-03-31
3,NaN,BARROW HIGH SCHOOL,BARROW,AK,12,11,2022/2023,2022-03-31
4,NaN,BARTLETT HIGH SCHOOL,ANCHORAGE,AK,41,37,2022/2023,2022-03-31


In [14]:
fafsa_with_codes = fafsa_raw[['Cycle','As of Date','Name','City','State','Submitted',
                                     'Completed']].sort_values(by = ['As of Date','State','City','Name']
                                                               , ascending = True)
fafsa_with_codes

,Cycle,As of Date,Name,City,State,Submitted,Completed
163679,2016/2017,2016-01-31,AKIACHAK SCHOOL,AKIACHAK,AK,<5,<5
163680,2016/2017,2016-01-31,ALAKANUK SCHOOL,ALAKANUK,AK,<5,<5
163682,2016/2017,2016-01-31,ALASKA MILITARY YOUTH ACADEMY,ANCHORAGE,AK,<5,<5
163683,2016/2017,2016-01-31,ANCHORAGE CHRISTIAN SCHOOLS,ANCHORAGE,AK,7,5
163685,2016/2017,2016-01-31,AVAIL SCHOOL,ANCHORAGE,AK,<5,<5
...,...,...,...,...,...,...,...
2582874,2023/2024,2023-12-31,UPTON HIGH SCHOOL,UPTON,WY,8,7
2582876,2023/2024,2023-12-31,WHEATLAND HIGH SCHOOL,WHEATLAND,WY,37,37
2582880,2023/2024,2023-12-31,WORLAND HIGH SCHOOL,WORLAND,WY,64,63
2582881,2023/2024,2023-12-31,WRIGHT JR. & SR. HIGH SCHOOL,WRIGHT,WY,14,13


Next cleanup: Submitted and Completed. We'll remove anything with '<5' in either column, putting a floor on school size in consideration. We'll also reduce to just WA. Then we'll update column types to ensure the numbers are numeric.

The <5 filter removes about 160k rows.

In [15]:
fafsa_clean = fafsa_with_codes[(fafsa_with_codes['Submitted'] != '<5') & (fafsa_with_codes['Completed'] != '<5')]

fafsa_wa = fafsa_clean[fafsa_clean['State'] == 'WA']

fafsa_wa['SchoolYear'] = fafsa_wa['Cycle'].str.slice(5,9)

fafsa_data = fafsa_wa.astype({'Submitted': int, 'Completed': int})

/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_56016/3170210233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fafsa_wa['SchoolYear'] = fafsa_wa['Cycle'].str.slice(5,9)


Let's add county to everything.

In [16]:
county = pd.read_csv('Counties.csv')
county['CITY NAME'] = county['CITY NAME'].str.upper()

fafsa_county = fafsa_data.merge(county, left_on = 'City', right_on = 'CITY NAME')

fafsa = fafsa_county[['SchoolYear','As of Date','Name','COUNTY NAME','Submitted','Completed']]
fafsa

,SchoolYear,As of Date,Name,COUNTY NAME,Submitted,Completed
0,2017,2016-01-31,J M WEATHERWAX HIGH SCHOOL,Grays Harbor,26,19
1,2017,2016-02-29,J M WEATHERWAX HIGH SCHOOL,Grays Harbor,39,33
2,2017,2016-03-31,HARBOR HIGH SCHOOL,Grays Harbor,6,5
3,2017,2016-03-31,J M WEATHERWAX HIGH SCHOOL,Grays Harbor,57,47
4,2017,2016-04-30,HARBOR HIGH SCHOOL,Grays Harbor,7,6
...,...,...,...,...,...,...
47450,2024,2023-08-31,PACIFIC CREST ONLINE ACADEMY,Benton,9,6
47451,2024,2023-09-30,PACIFIC CREST ONLINE ACADEMY,Benton,9,7
47452,2024,2023-10-31,PACIFIC CREST ONLINE ACADEMY,Benton,9,7
47453,2024,2023-11-30,PACIFIC CREST ONLINE ACADEMY,Benton,9,7


In [17]:
#fafsa[(fafsa['COUNTY NAME'] == 'Thurston') & (fafsa['SchoolYear'] == '2023') & (fafsa['As of Date'] == '2022-12-31')]

## Graduation Rates
Now pull in graduation data. This data hasn't been combined yet, so we'll do that here.

In [18]:
# get all the grad files in the folder
all_files = glob.glob('Graduation raw/*.csv')

li = []

# unpack the list of files
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, dtype = object)
    li.append(df)

# put the files all together into one df
grad_raw = pd.concat(li, axis=0, ignore_index=True)
grad_raw.head()

,SchoolYear,OrganizationLevel,County,ESDName,ESDOrganizationId,DistrictCode,DistrictName,DistrictOrganizationId,SchoolCode,SchoolName,...,Year6Dropout,Year7Dropout,TransferOut,FinalCohort,Graduate,Continuing,Dropout,GraduationRate,DataAsOf,CurrentSchoolType
0,2023,School,Grays Harbor,Capital Region ESD 113,100004,14005,Aberdeen School District,100010,5514,Grays Harbor Academy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/11/2023 12:00:00 AM,NaN
1,2023,School,Grays Harbor,Capital Region ESD 113,100004,14005,Aberdeen School District,100010,5514,Grays Harbor Academy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/11/2023 12:00:00 AM,NaN
2,2023,School,Grays Harbor,Capital Region ESD 113,100004,14005,Aberdeen School District,100010,5514,Grays Harbor Academy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/11/2023 12:00:00 AM,NaN
3,2023,School,Grays Harbor,Capital Region ESD 113,100004,14005,Aberdeen School District,100010,5514,Grays Harbor Academy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/11/2023 12:00:00 AM,NaN
4,2023,School,Grays Harbor,Capital Region ESD 113,100004,14005,Aberdeen School District,100010,5514,Grays Harbor Academy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/11/2023 12:00:00 AM,NaN


In [25]:
# Filter the data to just school level data for all students
grad_filter1 = grad_raw[(grad_raw['OrganizationLevel'] == 'School') & (grad_raw['StudentGroupType'] == 'All')]

# We also filter out the records with surpressed data. This is a known limitation: small groups
# or groups that could possibly be traced are filtered out upstream in the data.
# Possible workaround: aggregate to district level

grad_filter2 = grad_filter1[(grad_filter1['Suppression'] == 'No Suppression') 
                            | (grad_filter1['Suppression'] == 'No DAT')]

# Finally, filter to just the 4-year cohort. If we had complete data, we would do a rolling calculation
# to include 'super seniors' in the graduation count each year... but with suppressed data, this is asking for 
# trouble

grad_filter3 = grad_filter2[grad_filter2['Cohort'] == 'Four Year']

# Pick the relevent columns

grad = grad_filter3[['SchoolYear','County','DistrictName','SchoolName','FinalCohort','Graduate']]
grad['SchoolName'] = grad['SchoolName'].str.upper()
grad

/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_56016/984140451.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grad['SchoolName'] = grad['SchoolName'].str.upper()


,SchoolYear,County,DistrictName,SchoolName,FinalCohort,Graduate
74,2023,Grays Harbor,Aberdeen School District,HARBOR JUNIOR/SENIOR HIGH SCHOOL,26,20
224,2023,Grays Harbor,Aberdeen School District,J M WEATHERWAX HIGH SCHOOL,191,154
394,2023,Skagit,Anacortes School District,ANACORTES HIGH SCHOOL,180,173
483,2023,Skagit,Anacortes School District,CAP SANTE HIGH SCHOOL,38,28
612,2023,Snohomish,Arlington School District,ARLINGTON HIGH SCHOOL,399,364
...,...,...,...,...,...,...
680021,2017,Yakima,Yakima School District,YAKIMA ONLINE,37,17
680184,2017,Yakima,Yakima School District,YAKIMA SATELLITE ALTERNATIVE PROGRAMS,22,3
680332,2017,Thurston,Yelm School District,YELM EXTENSION SCHOOL,67,17
680413,2017,Thurston,Yelm School District,YELM HIGH SCHOOL 12,369,320


We'll be joining based on school name, county, and school year.

In [26]:
joined = fafsa.merge(grad, left_on = ['SchoolYear','Name','COUNTY NAME'], right_on = ['SchoolYear','SchoolName','County'])
output = joined[['SchoolYear','As of Date','Name','DistrictName','County','FinalCohort','Graduate','Submitted','Completed']]
output

,SchoolYear,As of Date,Name,DistrictName,County,FinalCohort,Graduate,Submitted,Completed
0,2017,2016-01-31,J M WEATHERWAX HIGH SCHOOL,Aberdeen School District,Grays Harbor,192,162,26,19
1,2017,2016-02-29,J M WEATHERWAX HIGH SCHOOL,Aberdeen School District,Grays Harbor,192,162,39,33
2,2017,2016-03-31,J M WEATHERWAX HIGH SCHOOL,Aberdeen School District,Grays Harbor,192,162,57,47
3,2017,2016-04-30,J M WEATHERWAX HIGH SCHOOL,Aberdeen School District,Grays Harbor,192,162,66,57
4,2017,2016-05-31,J M WEATHERWAX HIGH SCHOOL,Aberdeen School District,Grays Harbor,192,162,70,65
...,...,...,...,...,...,...,...,...,...
25671,2022,2021-08-31,TWO RIVERS SCHOOL,Snoqualmie Valley School District,King,11,9,7,5
25672,2022,2021-09-30,TWO RIVERS SCHOOL,Snoqualmie Valley School District,King,11,9,7,5
25673,2022,2021-10-31,TWO RIVERS SCHOOL,Snoqualmie Valley School District,King,11,9,8,6
25674,2022,2021-11-30,TWO RIVERS SCHOOL,Snoqualmie Valley School District,King,11,9,8,6


In [27]:
output.to_csv('FAFSA Graduation Data.csv')

## Kindergarten Readiness

In [23]:
# get all the grad files in the folder
all_files = glob.glob('Kindergarten Readiness/*.csv')

li = []

# unpack the list of files
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, dtype = object)
    li.append(df)

# put the files all together into one df
kind_raw = pd.concat(li, axis=0, ignore_index=True)
kind_raw.head()

,schoolyear,OrganizationLevel,OrganizationId,WashingtonStateCode,washingtonstatename,ESDOrganizationId,ESDName,DistrictOrganizationId,DistrictName,SchoolOrganizationId,...,StudentGroup,Domain,Measure,MeasureValue,Numerator,Denominator,Percent,DevelopmentLevel,Suppress,DataAsOf
0,2014-15,District,100010,103300,State Total,100004,Capital Region ESD 113,100010,Aberdeen School District,NaN,...,Asian,Math,MathReadinessFlag,Y,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-15,District,100010,103300,State Total,100004,Capital Region ESD 113,100010,Aberdeen School District,NaN,...,Low-Income,Math,MathReadinessFlag,Y,120,191,0.62827,NaN,NaN,NaN
2,2014-15,District,100010,103300,State Total,100004,Capital Region ESD 113,100010,Aberdeen School District,NaN,...,Male,Math,MathDevelopmentLevel,3-year olds,26,138,0.18841,NaN,NaN,NaN
3,2014-15,District,100010,103300,State Total,100004,Capital Region ESD 113,100010,Aberdeen School District,NaN,...,Male,Math,MathDevelopmentLevel,4-year olds,51,138,0.36957,NaN,NaN,NaN
4,2014-15,District,100010,103300,State Total,100004,Capital Region ESD 113,100010,Aberdeen School District,NaN,...,Female,Physical,PhysicalReadinessFlag,N,8,122,0.06557,NaN,NaN,NaN


In [24]:
# Filter the data to just school level data for all students
kind_filter1 = kind_raw[(kind_raw['OrganizationLevel'] == 'School') & (kind_raw['StudentGroupType'] == 'All')]

kind_filter2 = kind_filter1[(kind_filter1['Measure'] == 'NumberofDomainsReadyforKindergarten') & (kind_filter1['MeasureValue'] == '6')]

# Pick the relevent columns
# Pick the relevent columns

kind = kind_filter2[['schoolyear','DistrictName','SchoolName','Percent']]
kind['SchoolName'] = kind['SchoolName'].str.upper()
kind


/var/folders/13/tb353hv16bg0qsrdngc6r2v40000gn/T/ipykernel_56016/2592029662.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kind['SchoolName'] = kind['SchoolName'].str.upper()


,schoolyear,DistrictName,SchoolName,Percent
102217,2014-15,Othello School District,LUTACAGA ELEMENTARY,0.05376
102738,2014-15,Othello School District,HIAWATHA ELEMENTARY SCHOOL,0.32558
103817,2014-15,Lind School District,LIND ELEMENTARY SCHOOL,0.92857
104093,2014-15,Clarkston School District,PARKWAY ELEMENTARY,0.42553
104609,2014-15,Clarkston School District,GRANTHAM ELEMENTARY,0.2439
...,...,...,...,...
7322890,2015-16,SOAR Academy Charter District,SOAR ACADEMY (CLOSED AFTER 2018-2019 SCHOOL YEAR),0.92857
7323075,2015-16,Clover Park School District,FOUR HEROES ELEMENTARY,0.42697
7323727,2015-16,Pasco School District,BARBARA MCCLINTOCK STEM ELEMENTARY,0.38095
7324380,2015-16,Pasco School District,CAPTAIN GRAY STEM ELEMENTARY,0.05263


In [36]:
# get unique list of districts and counties

dlist = output[['DistrictName','County']].drop_duplicates()
dlist

,DistrictName,County
0,Aberdeen School District,Grays Harbor
122,Anacortes School District,Skagit
231,Arlington School District,Snohomish
243,Lakewood School District,Snohomish
414,Asotin-Anatone School District,Asotin
...,...,...
25389,Morton School District,Lewis
25458,Mossyrock School District,Lewis
25504,Tekoa School District,Whitman
25550,Oakville School District,Grays Harbor


In [38]:
kind_joined = kind.merge(dlist, left_on = ['DistrictName'], right_on = ['DistrictName'], how = 'left')
kind_output = kind_joined[['schoolyear','County','DistrictName','SchoolName','Percent']]
kind_output

,schoolyear,County,DistrictName,SchoolName,Percent
0,2014-15,Adams,Othello School District,LUTACAGA ELEMENTARY,0.05376
1,2014-15,Adams,Othello School District,HIAWATHA ELEMENTARY SCHOOL,0.32558
2,2014-15,Adams,Lind School District,LIND ELEMENTARY SCHOOL,0.92857
3,2014-15,Asotin,Clarkston School District,PARKWAY ELEMENTARY,0.42553
4,2014-15,Asotin,Clarkston School District,GRANTHAM ELEMENTARY,0.2439
...,...,...,...,...,...
10255,2015-16,NaN,SOAR Academy Charter District,SOAR ACADEMY (CLOSED AFTER 2018-2019 SCHOOL YEAR),0.92857
10256,2015-16,Pierce,Clover Park School District,FOUR HEROES ELEMENTARY,0.42697
10257,2015-16,Franklin,Pasco School District,BARBARA MCCLINTOCK STEM ELEMENTARY,0.38095
10258,2015-16,Franklin,Pasco School District,CAPTAIN GRAY STEM ELEMENTARY,0.05263


In [39]:
kind_output.to_csv('Kindergarten.csv')

## ALICE

In [40]:
alice_raw = pd.read_excel('ALICE/DataSheet_WA.xlsx', sheet_name = 'County')
alice_raw.head()

,Year,GEO.id2,GEO.display_label,County,State,State Abbr,Households,Poverty Households,ALICE Households,Above ALICE Households,ALICE Threshold - HH under 65,ALICE Threshold - HH 65 years and over,Source: American Community Survey
0,2010,53001,"Adams County, Washington",Adams,Washington,WA,5599,1044,1504,3051,40000,30000,5-Year
1,2012,53001,"Adams County, Washington",Adams,Washington,WA,5722,1117,1806,2799,45000,30000,5-Year
2,2014,53001,"Adams County, Washington",Adams,Washington,WA,5827,1166,1833,2828,50000,30000,5-Year
3,2016,53001,"Adams County, Washington",Adams,Washington,WA,5733,1076,1722,2935,50000,35000,5-Year
4,2018,53001,"Adams County, Washington",Adams,Washington,WA,5881,1135,1733,3013,50000,40000,5-Year


In [42]:
# Pick the relevent columns

alice = alice_raw[['Year','County','Households','Poverty Households','ALICE Households']]
alice

,Year,County,Households,Poverty Households,ALICE Households
0,2010,Adams,5599,1044,1504
1,2012,Adams,5722,1117,1806
2,2014,Adams,5827,1166,1833
3,2016,Adams,5733,1076,1722
4,2018,Adams,5881,1135,1733
...,...,...,...,...,...
268,2014,Yakima,79700,14271,22649
269,2016,Yakima,81084,12279,25681
270,2018,Yakima,83320,12963,24844
271,2019,Yakima,83992,12400,23232


In [43]:
alice.to_csv('ALICE.csv')